In [15]:
import configparser
from glob import glob
import json
from datetime import datetime, date, timezone, timedelta
from time import sleep

In [2]:
stamp_path = r'.//scraping//instagram//stamps.ini'
temp_storage = r'./scraping/data/temp_storage/'
storage = r'./scraping/data/storage/'

In [9]:
config = configparser.ConfigParser()
config.read(stamp_path)
for key in config['users']:
    print(key, config['users'][key])

hr3 1577833200
bayerischer_rundfunk 1577833200
mdr_th 1577833200
mdr_san 1577833200
mdr_sn 1577833200
wdr2 1577833200
tagesschau 1577833200
welt 1577833200
spiegelmagazin 1577833200
ndr2 1577833200
ndrhamburg 1577833200
ndrniedersachsen 1577833200
swraktuell 1577833200
swr3online 1577833200
faz 1577833200


In [52]:
def append_to_json(new_data, file):
    
    def write_json(data, file):
        with open(file,'w') as f:
            json.dump(data, f, indent=4)
            f.close()
    
    try:
        with open(file) as json_file:
            data = json.load(json_file)

            data += new_data

            json_file.close()
    except FileNotFoundError:
        data = new_data

    try:
        write_json(data, file)
    except OSError:
        sleep(0.2)
        write_json(data, file)

In [55]:
temp_files = glob(temp_storage + "*.txt")
for j in temp_files:
    
    f = open(j, mode="r", encoding="utf-8")
    
    dic_collection = {}
    
    for i in f:
        
        temp = json.loads(i[:-1])
        s = temp['date']
        dt = datetime(int(s[:4]), int(s[5:7]), int(s[8:10]), int(s[11:13]), int(s[14:16]), int(s[17:19])) # String to datetime.date
        if dt.year < 2020:
            print(dt)
            break
            
        dic = {}
        
        dic['date'] = int(dt.timestamp())
        dic['text'] = temp['content']
        dic['user'] = temp['user']['username']
        dic['shortcode'] = temp['id']
        dic['platform'] = 'twitter'
        dic['likes'] = temp['likeCount']
        
        year = str(dt.year)
        month = str(dt.month)
        
        if year not in dic_collection:
            dic_collection[year] = {}
        if month not in dic_collection[year]:
            dic_collection[year][month] = []
        
        dic_collection[year][month].append(dic)

    # Closing file 
    f.close()
    
    for year in dic_collection:
        for month in dic_collection[year]:
            path = storage+year+'_'+month+'.json'
            append_to_json(dic_collection[year][month], path)
    
    print('Completed:', j)

Completed: ./scraping/data/temp_storage\BR_Presse.txt
Completed: ./scraping/data/temp_storage\derspiegel.txt
Completed: ./scraping/data/temp_storage\faznet.txt
Completed: ./scraping/data/temp_storage\hessenschau.txt
Completed: ./scraping/data/temp_storage\hrPresse.txt
Completed: ./scraping/data/temp_storage\MDRAktuell.txt
Completed: ./scraping/data/temp_storage\mdrde.txt
Completed: ./scraping/data/temp_storage\MDR_SAN.txt
Completed: ./scraping/data/temp_storage\MDR_SN.txt
Completed: ./scraping/data/temp_storage\mdr_th.txt
Completed: ./scraping/data/temp_storage\ndr.txt
Completed: ./scraping/data/temp_storage\NDRinfo.txt
Completed: ./scraping/data/temp_storage\swr3.txt
Completed: ./scraping/data/temp_storage\SWRAktuellBW.txt
Completed: ./scraping/data/temp_storage\SZ.txt
Completed: ./scraping/data/temp_storage\tagesschau.txt
Completed: ./scraping/data/temp_storage\WDR.txt
Completed: ./scraping/data/temp_storage\WDR2.txt
Completed: ./scraping/data/temp_storage\WDRaktuell.txt
Completed: .